TP intégration des données

In [3]:
import sys
from pathlib import Path
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import sys
from pathlib import Path

notebook_dir = Path().resolve()
project_root = notebook_dir.parent

sys.path.insert(0, project_root)
print(f"✅ Chemin ajouté : {project_root}")

✅ Chemin ajouté : /Users/vitorpinto/Documents/EPSI/Atelier Integration des donnees/AID_10_12_2025/TP/Music-Analysis-ETL


Lecture de la source

In [4]:
df_low_popularity = pd.read_csv('../data/raw/low_popularity_spotify_data.csv')


Information

In [5]:
print("Premières 10 valeurs\n")
print(df_low_popularity.head(10))

print("\nDescription et quelques métriques\n")
print(df_low_popularity.describe())

print("\nTypes des données par colonne\n")
print(df_low_popularity.info())




Premières 10 valeurs

   time_signature  track_popularity  speechiness  danceability  playlist_name  \
0             4.0                23       0.0393         0.636  Rock Classics   
1             4.0                53       0.0317         0.572  Rock Classics   
2             4.0                55       0.0454         0.591  Rock Classics   
3             4.0                64       0.1010         0.443  Jazz Classics   
4             4.0                62       0.0298         0.685  Jazz Classics   
5             4.0                61       0.0566         0.670  Jazz Classics   
6             4.0                60       0.0914         0.663  Jazz Classics   
7             4.0                55       0.0373         0.525  Jazz Classics   
8             4.0                54       0.0363         0.336  Jazz Classics   
9             4.0                53       0.0407         0.742  Jazz Classics   

                       track_artist  duration_ms  energy playlist_genre  \
0      Cree

Valeurs nulles

In [6]:
print("Valeurs nulles par colonne :\n")
print(df_low_popularity.isnull().sum())

print("\nTotal des valeurs nulles :\n")
print(df_low_popularity.isnull().sum().sum())


Valeurs nulles par colonne :

time_signature              1
track_popularity            0
speechiness                 1
danceability                1
playlist_name               0
track_artist                0
duration_ms                 1
energy                      1
playlist_genre              0
playlist_subgenre           0
track_href                  1
track_name                  0
mode                        1
uri                         1
type                        1
track_album_release_date    0
analysis_url                1
id                          1
instrumentalness            1
track_album_id              0
playlist_id                 0
track_id                    0
valence                     1
key                         1
tempo                       1
loudness                    1
acousticness                1
liveness                    1
track_album_name            0
dtype: int64

Total des valeurs nulles :

18


Valeurs negatives

In [7]:
numeric_cols = df_low_popularity.select_dtypes(include=['float64', 'int64']).columns

negatives = df_low_popularity[numeric_cols] < 0

print("Nombre de valeurs négatives par colonne :")
for col in numeric_cols:
    total_neg = negatives[col].sum()
    if total_neg > 0:
        print(f"{col}: {total_neg} valeurs négatives")
print("\nDonnées correctes si seulement aparaitre la colonne loudness")

Nombre de valeurs négatives par colonne :
loudness: 3139 valeurs négatives

Données correctes si seulement aparaitre la colonne loudness


Savoir si la colonne "analysis_url" contient que des urls

In [8]:
url_pattern = re.compile(
    r'^(https?://)?'
    r'([a-zA-Z0-9.-]+)?'
    r'(\.[a-zA-Z]{2,})'
    r'(/[\w./?%&=-]*)?$'
)


df_low_popularity['is_valid_url'] = df_low_popularity['analysis_url'].apply(lambda x: bool(url_pattern.match(str(x))))

invalid_urls = df_low_popularity[df_low_popularity['is_valid_url'] == False]['analysis_url']
print(invalid_urls)

nan_urls = df_low_popularity[df_low_popularity['analysis_url'].isna()]
print(nan_urls)

263    NaN
Name: analysis_url, dtype: object
     time_signature  track_popularity  speechiness  danceability  \
263             NaN                50          NaN           NaN   

              playlist_name track_artist  duration_ms  energy playlist_genre  \
263  Chill Lofi Study Beats     Berhanio          NaN     NaN           lofi   

    playlist_subgenre  ...             playlist_id                track_id  \
263             chill  ...  37i9dQZF1DX8Uebhn9wzrS  2teI76KKFE6qkpLZJs7tZ7   

     valence key tempo loudness acousticness liveness  track_album_name  \
263      NaN NaN   NaN      NaN          NaN      NaN           Make It   

    is_valid_url  
263        False  

[1 rows x 30 columns]


Dates invalides

In [9]:
df_low_popularity['release_date_parsed'] = pd.to_datetime(df_low_popularity['track_album_release_date'], format='%Y-%m-%d', errors='coerce')

total_invalid_dates = df_low_popularity['release_date_parsed'].isna().sum()
print("Nombre total de dates invalides ou mal formatées :", total_invalid_dates)

Nombre total de dates invalides ou mal formatées : 57


Taux de completude

In [10]:
taux_completude = df_low_popularity.notna().sum() / len(df_low_popularity) * 100
print("\nTaux de complétude :\n")
print(taux_completude)



Taux de complétude :

time_signature               99.968203
track_popularity            100.000000
speechiness                  99.968203
danceability                 99.968203
playlist_name               100.000000
track_artist                100.000000
duration_ms                  99.968203
energy                       99.968203
playlist_genre              100.000000
playlist_subgenre           100.000000
track_href                   99.968203
track_name                  100.000000
mode                         99.968203
uri                          99.968203
type                         99.968203
track_album_release_date    100.000000
analysis_url                 99.968203
id                           99.968203
instrumentalness             99.968203
track_album_id              100.000000
playlist_id                 100.000000
track_id                    100.000000
valence                      99.968203
key                          99.968203
tempo                        99.968203
lo

Taux de coherence

In [11]:
date_cols = ['track_album_release_date']
numeric_cols = ['time_signature','track_popularity','speechiness','danceability',
                'duration_ms','energy','mode','instrumentalness','valence',
                'key','tempo','loudness','acousticness','liveness']
text_cols = ['playlist_name','track_artist','playlist_genre','playlist_subgenre',
             'track_name','track_album_name','track_album_id','playlist_id',
             'track_id','uri','type','track_href','analysis_url','id']

total_rows = len(df_low_popularity)
total_coherence_sum = 0
total_columns = len(date_cols + numeric_cols + text_cols)

coherence_per_column = []

for col in date_cols + numeric_cols + text_cols:
    if col in date_cols:
        coherence = pd.to_datetime(df_low_popularity[col], format='%Y-%m-%d', errors='coerce').notna().sum() / total_rows
    else:
        coherence = df_low_popularity[col].notna().sum() / total_rows
    total_coherence_sum += coherence
    coherence_per_column.append((col, round(coherence*100, 2)))

total_coherence = total_coherence_sum / total_columns * 100

print(f"Taux de cohérence total : {total_coherence:.2f}%\n")
print("Cohérence par colonne :")
for col_name, coh in coherence_per_column:
    print(f"{col_name}: {coh}%")


Taux de cohérence total : 99.92%

Cohérence par colonne :
track_album_release_date: 98.19%
time_signature: 99.97%
track_popularity: 100.0%
speechiness: 99.97%
danceability: 99.97%
duration_ms: 99.97%
energy: 99.97%
mode: 99.97%
instrumentalness: 99.97%
valence: 99.97%
key: 99.97%
tempo: 99.97%
loudness: 99.97%
acousticness: 99.97%
liveness: 99.97%
playlist_name: 100.0%
track_artist: 100.0%
playlist_genre: 100.0%
playlist_subgenre: 100.0%
track_name: 100.0%
track_album_name: 100.0%
track_album_id: 100.0%
playlist_id: 100.0%
track_id: 100.0%
uri: 99.97%
type: 99.97%
track_href: 99.97%
analysis_url: 99.97%
id: 99.97%


Taux d'erreur

In [12]:
total_error = 100 - total_coherence
print(f"Taux d'erreur total : {total_error:.2f}%\n")

print("Taux d'erreur par colonne :\n")
for col_name, coh in coherence_per_column:
    print(f"{col_name}: {100 - coh:.2f}%")


Taux d'erreur total : 0.08%

Taux d'erreur par colonne :

track_album_release_date: 1.81%
time_signature: 0.03%
track_popularity: 0.00%
speechiness: 0.03%
danceability: 0.03%
duration_ms: 0.03%
energy: 0.03%
mode: 0.03%
instrumentalness: 0.03%
valence: 0.03%
key: 0.03%
tempo: 0.03%
loudness: 0.03%
acousticness: 0.03%
liveness: 0.03%
playlist_name: 0.00%
track_artist: 0.00%
playlist_genre: 0.00%
playlist_subgenre: 0.00%
track_name: 0.00%
track_album_name: 0.00%
track_album_id: 0.00%
playlist_id: 0.00%
track_id: 0.00%
uri: 0.03%
type: 0.03%
track_href: 0.03%
analysis_url: 0.03%
id: 0.03%


Quels genres et artistes dominent le classement ?

In [13]:
top_genres_pop = df_low_popularity.groupby('playlist_genre')['track_popularity'].mean().sort_values(ascending=False).head(10)
print("Top 10 genres les plus populaires (en moyenne) :\n")
print(top_genres_pop)

top_artists_pop = df_low_popularity.groupby('track_artist')['track_popularity'].mean().sort_values(ascending=False).head(10)
print("\nTop 10 artistes les plus populaires (en moyenne) :\n")
print(top_artists_pop)

Top 10 genres les plus populaires (en moyenne) :

playlist_genre
korean      59.769231
metal       59.071429
j-pop       58.833333
indie       58.307692
k-pop       56.500000
reggae      55.578947
mandopop    53.785714
punk        52.625000
soul        51.619048
wellness    49.512500
Name: track_popularity, dtype: float64

Top 10 artistes les plus populaires (en moyenne) :

track_artist
Jasleen Royal, B Praak, Romy, Anvita Dutt              68.0
Dimitri Vegas, Steve Aoki, Ann Lee, Sound Of Legend    68.0
Youka                                                  68.0
Shubh, Ikky                                            68.0
Don Omar, Tego Calderón                                68.0
Polo G                                                 68.0
Sukha, Chani Nattan, Prodgk, Gurlez Akhtar             68.0
Zaz                                                    68.0
Lost Frequencies                                       68.0
Rotimi, Wale                                           68.0
Name: trac

Comment la durée ou le contenu explicite d’un titre impacte sa popularité ?

In [14]:
df_low_popularity['duration_min'] = df_low_popularity['duration_ms'] / 60000

corr_duration = df_low_popularity[['duration_min','track_popularity']].corr().iloc[0,1]
print(f"Corrélation durée vs popularité : {corr_duration:.2f}")

Corrélation durée vs popularité : -0.06


Quelles sont les décennies les plus ‘populaires’ selon les données ?

In [15]:
df_low_popularity['release_year'] = pd.to_datetime(df_low_popularity['track_album_release_date'], errors='coerce').dt.year

df_low_popularity['decade'] = (df_low_popularity['release_year'] // 10) * 10

decade_pop = df_low_popularity.groupby('decade')['track_popularity'].mean().sort_values(ascending=False)

top3_decades = decade_pop.head(3)
print("Top 3 des décennies les plus populaires :\n")
print(top3_decades)


Top 3 des décennies les plus populaires :

decade
1990.0    48.161017
2020.0    44.031753
2000.0    42.325000
Name: track_popularity, dtype: float64
